In [3]:
import tqdm
import tensorflow as tf
import numpy as np
import tqdm
import pickle
import numpy
import os
import csv
from mittens import Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from scipy import spatial 
%run constants.py
tf.random.set_seed(
    SEED
)
np.random.seed(SEED)


In [6]:
with open(TRAIN_CORPORA, "rb") as f:
        train_corpora = pickle.load(f)
train_corpora


['uper',
 '2-10',
 'Andrews',
 'Mitchell',
 'Synulox',
 '180',
 'quickly',
 'single-Pack',
 'sardines',
 'cvd',
 'guage',
 'mild-moderate',
 'months)',
 'trizaural',
 '(10-25kg)',
 'Swann-Morton',
 'healead',
 '(10',
 '102.',
 'Travelling',
 'Gran',
 '40min',
 'August.',
 'Crossbreed',
 'Clinic-Blood',
 'Scrotum',
 'dander',
 'Oxygen',
 'facial',
 'Vitamin',
 'KK',
 'decison,',
 'nail.',
 'Related',
 'Poppy',
 'bills,',
 'dirnking',
 'coat-ok',
 'non-regen',
 'susceptibility',
 'exciting',
 'Siedlecki',
 'body-',
 'sutrgical',
 '<b>To',
 'nostrils,',
 'intra-cranial',
 'a119d02',
 'consistently',
 'Taking',
 'bouncy.',
 'infor',
 'begining',
 'vitsla',
 'LAME',
 'Ava',
 'serous',
 'Reaccess',
 'relaxeda',
 '1"',
 'Tied',
 'Side',
 'mobile',
 'BILATERL',
 'In',
 'light,',
 '18.9',
 '01-2021)',
 'A108A02',
 'clips',
 'suddenly',
 'polished.',
 'ears/teeth',
 'Omega',
 'Charlotte*',
 'Haematocrit',
 'PRIMARY',
 'Fed',
 'Preanesthetic',
 'matches',
 'introduction',
 'deowrm',
 'PC',
 'refl

In [6]:
def get_embedding_index():
    embeddings_index = {}

    f = open(GLOVE_100D_FILEPATH)
    for line in tqdm.tqdm(f, ncols=100, desc="Loading Glove Embeddings."):
        values = line.split()
        word = values[0]
        coefs = numpy.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs
    f.close()

    return embeddings_index

In [7]:
ei = get_embedding_index()

Loading Glove Embeddings.: 400000it [00:05, 68167.67it/s]


In [1]:
train_corpora

NameError: name 'train_corpora' is not defined

In [9]:
# flat_list_train_corpora = [x for xs in train_corpora for x in xs]
flat_list_train_corpora = list(set(train_corpora))
flat_list_train_corpora

# a = [flat_list_train_corpora.lower() for x in flat_list_train_corpora]
# a

['ordered',
 'using-',
 'Bumps',
 'assistance',
 'QOL',
 'vitsla',
 '>30BPM',
 'Caudodorsal',
 '£109,',
 'apptm',
 'teeh',
 'URT.',
 'instruction',
 'petalexin.',
 'h/r',
 'M/chip',
 '(4-10kg)-Pack',
 'healed,',
 'jm',
 'Ear-0.5',
 'scooting',
 'helaing',
 'disease/diet',
 '19/09/2019',
 'Pupil',
 'liquid,',
 'tomorr,',
 '600',
 'antirobe,',
 '-eyes',
 'daughter',
 'lump</b>',
 'score',
 '?less',
 'eod.',
 'within',
 'cyst?',
 'CON',
 'scrapes',
 'theses',
 'PLN,',
 '(suspect',
 '(would',
 'MPS',
 'digging',
 '5.4mg',
 'v+/d+.',
 'allergin',
 'ytd',
 'underweight,',
 'pone',
 'Canaural',
 'UCU:',
 'test,',
 'vommiting.',
 'infection?',
 'EMR',
 'beh,',
 'idiopathic',
 'lameness.',
 '3.5ml',
 'Spey',
 'loose?',
 '(about',
 'triggers,',
 'urination',
 'Doingwell,',
 'Fluid',
 'eyes-',
 'SCVS',
 'coughing/sneezing.',
 'scrotal.',
 'aok',
 '(chanelle)',
 'Wharton',
 'ready',
 'Mack',
 'hibi,',
 'REPEAT',
 'Rest',
 'cyst-type',
 'express,',
 'Sept,',
 'fp3',
 'abx',
 'Check</b>',
 'monday-i

In [10]:

def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed


pre_glove = glove2dict(GLOVE_100D_FILEPATH)
sw = list(_stop_words.ENGLISH_STOP_WORDS)
train_corpora_for_glove = [token.lower() for token in flat_list_train_corpora if (token.lower() not in sw)]
oov = [token for token in train_corpora_for_glove if token not in pre_glove.keys()]

corp_vocab = list(set(oov))
brown_doc = [' '.join(train_corpora_for_glove)]

cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

mittens_model = Mittens(n=EMBEDDING_DIM, max_iter=10000)

new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

newglove = dict(zip(corp_vocab, new_embeddings))


/home/aaron/anaconda3/envs/tf/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
2022-07-24 16:04:16.964419: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2022-07-24 16:04:17.780382: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4 MB memory:  -> device: 0, name: GeForce RTX 2070, pci bus id: 0000:03:00.0, compute capability: 7.5
2022-07-24 16:04:17.781215: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6157 MB memory:  -> device: 1, name: GeForce RTX 2070, pci bus id: 0000:04:00.0, compute capability: 7.5
2022-07-24 16:04:17.970982: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-07-24 16:04:28.018198: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.38MiB (rounded to 6688512)requested by op W/Initializer/initial_value
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allo

ResourceExhaustedError: Graph execution error:

Detected at node 'W/Initializer/initial_value' defined at (most recent call last):
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/aaron/anaconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/a

In [ ]:
new_glove_merged = newglove | ei
len(new_glove_merged)

In [ ]:
try:
    os.remove(FINE_TUNED_GLOVE_300D_FILEPATH)
except OSError:
    pass
with open(FINE_TUNED_GLOVE_300D_FILEPATH, "w") as file:
    for k, v in new_glove_merged.items():
        dimensions = ""
        for x in v:
            dimensions += str(x) + " "
        glove_line = k + " " + dimensions
        file.write(glove_line)

In [ ]:

def find_closest_embeddings(embedding_index, embedding): 
    return sorted(embedding_index.keys(), key=lambda word: spatial.distance.euclidean(embedding_index[word], embedding))

In [ ]:
find_closest_embeddings(new_glove_merged, new_glove_merged['libstdc'])[0:10]

In [ ]:
#Should fail - as libstdc is NOT in the original glove matrix
find_closest_embeddings(ei, ei['libstdc'])[0:10]